In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [3]:
match_folder=np.loadtxt('/home/sumins/workspace/washout classification/base_data/condition_match_folder.txt',dtype=str)
match_folder=match_folder.tolist()

In [4]:
import ast
range_path='/home/sumins/workspace/washout classification/base_data/liverrange.txt'
range_value_path='/home/sumins/workspace/washout classification/base_data/liverrange_value.txt'

liverrange=[]
liverrange_value=[]
with open (range_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange.append(ast.literal_eval(line))

with open (range_value_path,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        liverrange_value.append(ast.literal_eval(line))

In [5]:
same=0
diff=0
diff_index=[]
for index, i in enumerate(liverrange_value):
    if i[0]==i[1]==i[2]==i[3]:
        same+=1
    else:
        diff+=1
        diff_index.append(index)
        

In [6]:
print("same ",same)
print("diff ", diff)

same  146
diff  19


In [7]:
for k,i in enumerate(diff_index):
    del match_folder[i-k]
    del liverrange[i-k]
    del liverrange_value[i-k]

In [8]:
Tumor_slice=[]
data_path='/disk1/HCC2'
for i in match_folder:    
    washout_dir = os.path.join(data_path, 'Anonymize_'+i,'washout_'+i+'.txt') 
    washout=[]
    with open(washout_dir,'r') as f:
        lines=f.readlines()
        slice=[]
        label=[]
        for line in lines:
            line=line.strip()
            washout.append(line)
        
        for i in [washout[0],washout[-1]]:
            k=i.split('\t')
            slice.append(int(k[0]))

    Tumor_slice.append(slice) 


In [9]:
import ast

path1='/home/sumins/workspace/washout classification/min_slice/arterial.txt'
path2='/home/sumins/workspace/washout classification/min_slice/pre.txt'
path3='/home/sumins/workspace/washout classification/min_slice/delay.txt'
path4='/home/sumins/workspace/washout classification/min_slice/portal.txt'

arterial=[]
pre=[]
delay=[]
portal=[]

with open (path1,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        arterial.append(ast.literal_eval(line))
with open (path2,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        pre.append(ast.literal_eval(line))
with open (path3,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        delay.append(ast.literal_eval(line))
with open (path4,'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        portal.append(ast.literal_eval(line))

In [10]:
for k,i in enumerate(diff_index):
    del arterial[i-k]
    del pre[i-k]
    del portal[i-k]
    del delay[i-k]

In [11]:
#for i in range(len(Tumor_slice)):
liver_label=[]

for j in range(len(liverrange)):
    liver_one=[]
    for i in range(liverrange[j][4],liverrange[j][5]+1):
        if i in range(Tumor_slice[j][0],Tumor_slice[j][1]+1):
            liver_one.append(1)
        else:
            liver_one.append(0)
    liver_label.append(liver_one)
        

In [12]:
Train_slice=liver_label[:120]
Test_slice=liver_label[120:]

In [13]:
Tumor=0
clear=0
for i in Train_slice:
    for j in i:
        if j==1:
            Tumor+=1
        else:
            clear+=1
print("Tumor: {}, Clear: {}".format(Tumor, clear))
            

Tumor: 1364, Clear: 3725


In [14]:
Tumor=0
clear=0
for i in Test_slice:
    for j in i:
        if j==1:
            Tumor+=1
        else:
            clear+=1
print("Tumor: {}, Clear: {}".format(Tumor, clear))

Tumor: 294, Clear: 783


In [19]:
A_DATA=[]
R_DATA=[]
P_DATA=[]   
D_DATA=[]
all_phase=[]
LABEL=[]
data_path='/disk1/HCC2'
for i in range(len(Test_slice)):
    totalnum=i+120
    folder_name=match_folder[totalnum]
    a_path=os.path.join(data_path, 'Anonymize_'+folder_name,'A_Anonymize_'+folder_name+'.nii') 
    a_data_all=nib.load(a_path).get_fdata()
    r_path=os.path.join(data_path, 'Anonymize_'+folder_name,'R_Anonymize_'+folder_name+'.nii') 
    r_data_all=nib.load(r_path).get_fdata()
    p_path=os.path.join(data_path, 'Anonymize_'+folder_name,'P_Anonymize_'+folder_name+'.nii') 
    p_data_all=nib.load(p_path).get_fdata()
    d_path=os.path.join(data_path, 'Anonymize_'+folder_name,'D_Anonymize_'+folder_name+'.nii') 
    d_data_all=nib.load(d_path).get_fdata()

    for j in range(0,liverrange_value[totalnum][0]):
        r=r_data_all[:,:,pre[totalnum][j]-1].astype(dtype='float32')
        a=a_data_all[:,:,arterial[totalnum][j]-1].astype(dtype='float32')
        p=p_data_all[:,:,portal[totalnum][j]-1].astype(dtype='float32')
        d=d_data_all[:,:,delay[totalnum][j]-1].astype(dtype='float32')
        DATASET=np.stack([r,a,p,d],axis=-1)
        all_phase.append(DATASET)

        LABEL.append(Test_slice[i][j])
    a_data_all=0
    r_data_all=0
    p_data_all=0
    d_data_all=0
    print(totalnum, "done")

120 done
121 done
122 done
123 done
124 done
125 done
126 done
127 done
128 done
129 done
130 done
131 done
132 done
133 done
134 done
135 done
136 done
137 done
138 done
139 done
140 done
141 done
142 done
143 done
144 done
145 done


In [20]:
k=np.array(all_phase)

data=nib.Nifti1Image(k,None)

In [21]:
nib.save(data,'/home/sumins/workspace/washout classification/transfer_learning/channel4_test.nii')

In [22]:
np.savetxt("/home/sumins/workspace/washout classification/transfer_learning/label_test.txt",LABEL,fmt='%d')